# Notebook to experiment with ways to quantify pec fin morphology

### Import packages

In [5]:
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import napari
import numpy as np
from napari_animation import Animation
import pandas as pd

### Now, load an image dataset along with nucleus masks inferred using cellpose.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# set parameters
filename = "2022_12_15 HCR Hand2 Tbx5a Fgf10a_1"
readPath = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/pecFin/HCR_Data/built_zarr_files/" + filename '_nucleus_props.csv'

image_df = pd.read_csv(readPath)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Mesh3d(x=image_df["X"], y=image_df["Y"], z=image_df["Z"],
                   alphahull=3,
                   opacity=0.8,
                   color='cyan')])
fig.show()

In [ ]:
from scipy.spatial import Delaunay
import numpy as np
from collections import defaultdict

pos = coords
alpha = 3

"""
Compute the alpha shape (concave hull) of a set of 3D points.
Parameters:
    pos - np.array of shape (n,3) points.
    alpha - alpha value.
return
    outer surface vertex indices, edge indices, and triangle indices
"""

tetra = Delaunay(pos)
# Find radius of the circumsphere.
# By definition, radius of the sphere fitting inside the tetrahedral needs 
# to be smaller than alpha value
# http://mathworld.wolfram.com/Circumsphere.html
tetrapos = np.take(pos,tetra.vertices,axis=0)
normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))

# Find tetrahedrals
tetras = tetra.vertices[r<alpha,:]
# triangles
TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
Triangles = tetras[:,TriComb].reshape(-1,3)
Triangles = np.sort(Triangles,axis=1)
# Remove triangles that occurs twice, because they are within shapes
TrianglesDict = defaultdict(int)
for tri in Triangles:TrianglesDict[tuple(tri)] += 1
Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
#edges
EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
Edges=Triangles[:,EdgeComb].reshape(-1,2)
Edges=np.sort(Edges,axis=1)
Edges=np.unique(Edges,axis=0)

In [ ]:
import math
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
from plotly.graph_objs import Mesh3d
import numpy as np

axis_lengths = np.array([11.89184616,  4.82438505,  1.49761732])

a = axis_lengths[2]
b = axis_lengths[1]
c = axis_lengths[0]

pi = math.pi
phi = np.linspace(0, 2*pi)
theta = np.linspace(-pi/2, pi/2)
phi, theta=np.meshgrid(phi, theta)
x = np.cos(theta) * np.sin(phi) * a
y = np.cos(theta) * np.cos(phi) * b
z = np.sin(theta) * c

# define a transformation matrix
R = np.array([[-0.23933507, -0.90749994,  0.34519933],
               [ 0.15574894,  0.31504453,  0.93621003],
               [ 0.95836371, -0.27783232, -0.06594095]])
T = np.zeros((4,4))
C = np.array([ 4.47448559, 192.68796498, 138.36717904])
T[0:3,0:3] = R
T[3,3] = 1
T[0:3, 3] = C

x=x.flatten() 
y=y.flatten() 
z=z.flatten()

xtf = []
ytf = []
ztf = []

for i in range(len(x)):
    xyz = np.array([z[i], y[i], x[i], 1]).reshape(4,1)
    xyz_tf = np.matmul(T,xyz)
    xyz_tf.flatten()
    xtf.append(xyz_tf[2][0])
    ytf.append(xyz_tf[1][0])
    ztf.append(xyz_tf[0][0])    

fig = go.Figure(data=[go.Mesh3d(x=xtf,
                                y=ytf,
                                z=ztf,
                                alphahull=0)])

# fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=4, range=[-30,30]+C[0],),
#                      yaxis = dict(nticks=4, range=[-30,30]+C[1],),
#                      zaxis = dict(nticks=4, range=[-30,30]+C[2],),))

fig.show()
